#uncomment to install llama-index, make sure it is above 0.10.x
## TODO :
Once the llama-index pip installable package is ready, implement the **models = NVIDIARerank.get_available_models()** , it is not implemnted right now !

## establish the reranker class with the local nim, supplying a base_url

In [1]:
from llama_index.postprocessor.nvidia_rerank import NVIDIARerank
rerank=NVIDIARerank().mode(type_mode="nim", base_url="http://localhost:1976/v1")

## we can set top_n like this

In [2]:
## setting top_n
rerank.top_n=2
print(rerank.top_n)
rerank.model.default, rerank.url

2


('nv-rerank-qa-mistral-4b:1', 'http://localhost:1976/v1/ranking')

## load some toy data

In [3]:
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.data_structs import Node
from llama_index.core.schema import NodeWithScore
query = "which way should i go?"
passages = [
    "two roads diverged in a yellow wood, and sorry i could not travel both and be one traveler, long i stood and looked down one as far as i could to where it bent in the undergrowth;",
    "then took the other, as just as fair, and having perhaps the better claim because it was grassy and wanted wear, though as for that the passing there had worn them really about the same,",
    "and both that morning equally lay in leaves no step had trodden black. oh, i marked the first for another day! yet knowing how way leads on to way i doubted if i should ever come back.",
    "i shall be telling this with a sigh somewhere ages and ages hense: two roads diverged in a wood, and i, i took the one less traveled by, and that has made all the difference."
]

nodes = [
    NodeWithScore(node=Node(text=passages[0]), score=0.7),
    NodeWithScore(node=Node(text=passages[1]), score=0.8),
]


## validate with NVreranker works

In [4]:
from llama_index.postprocessor.nvidia_rerank import NVIDIARerank
rerank.postprocess_nodes(nodes, query_str=query )


model nv-rerank-qa-mistral-4b:1 , url :http://localhost:1976/v1/ranking
----------> 
 [{'index': 0, 'logit': 1.185546875}, {'index': 1, 'logit': -0.783203125}]


[NodeWithScore(node=TextNode(id_='b8a03603-bd92-49d1-92a8-0f76db090f9c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='then took the other, as just as fair, and having perhaps the better claim because it was grassy and wanted wear, though as for that the passing there had worn them really about the same,', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.8),
 NodeWithScore(node=TextNode(id_='bdec85b9-570c-4a26-9e2f-dafecae7e46e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='two roads diverged in a yellow wood, and sorry i could not travel both and be one traveler, long i stood and looked down one as far as i could to where it bent in the undergrowth;', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metad

## switch to APICatalog , for that we need to set the NVIDIA_API_KEY

In [5]:
import getpass
import os

## API Key can be found by going to NVIDIA NGC -> AI Foundation Models -> (some model) -> Get API Code or similar.
## 10K free queries to any endpoint (which is a lot actually).

# del os.environ['NVIDIA_API_KEY']  ## delete key and reset
if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
    nvapi_key=os.environ["NVIDIA_API_KEY"] 

else:
    nvapi_key = getpass.getpass("NVAPI Key (starts with nvapi-): ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key


NVAPI Key (starts with nvapi-):  ······································································


In [6]:
from llama_index.postprocessor.nvidia_rerank import NVIDIARerank

rerank=NVIDIARerank().mode(type_mode="catalog")

In [7]:
rerank.model.default , rerank.url[0].default

('nv-rerank-qa-mistral-4b:1',
 'https://ai.api.nvidia.com/v1/retrieval/nvidia/reranking')

In [8]:
rerank.postprocess_nodes(nodes, query_str=query )

model nv-rerank-qa-mistral-4b:1 , url :https://ai.api.nvidia.com/v1/retrieval/nvidia/reranking
----------> 
 [{'index': 0, 'logit': -0.47705078125}, {'index': 1, 'logit': -7.921875}]
Query unsuccessful 200, please visit this page for more info : https://developer.nvidia.com/docs/nemo-microservices/inference/tools.html


[NodeWithScore(node=TextNode(id_='bdec85b9-570c-4a26-9e2f-dafecae7e46e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='two roads diverged in a yellow wood, and sorry i could not travel both and be one traveler, long i stood and looked down one as far as i could to where it bent in the undergrowth;', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=-0.47705078125),
 NodeWithScore(node=TextNode(id_='b8a03603-bd92-49d1-92a8-0f76db090f9c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='then took the other, as just as fair, and having perhaps the better claim because it was grassy and wanted wear, though as for that the passing there had worn them really about the same,', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{conte